In [2]:
%pip install lxml
#%pip install rdflib
import os
import s3fs
import gzip
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd

os.environ["AWS_ACCESS_KEY_ID"] = 'HKDQPOK3MAYOO8XODIKI'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'KmfYCYRdfwiNtp3ZvJAR6rm+giN15FY0cDW1KWqb'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJIS0RRUE9LM01BWU9POFhPRElLSSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzYyMzMxMjgxLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzYzMzY0NzExLCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc2Mjc1OTkxMSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDozNWQxY2YxNC0xYzQzLWU2YzItYjM2NS0xODBlM2M5M2IwNjUiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6ImNkZjM5NGE5LWQzZTMtNGZjZC05Y2E0LWUyNTU0ZTk5ODIwMyIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.Fllib9SFwFP4DQTMHiApv7lk-2SrQzwKLWthC5LtvlG3-W-2ZdGt2tCQ4M6Vjvf3yIgU4clB9QT59KTC3NJwuQ'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

path = "lab/dblp (1).xml.gz"



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.6 MB/s  0:00:00

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
articles = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(gz, events=("end",), tag="article", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("year")
            doi = elem.findtext("ee")

            # Collect all authors and their IDs
            authors, dblp, isnis, orcids, viafs, orkgs, gnds, wikis = [], [], [], [], [], [], [], []

            for a in elem.findall(".//author"):
                name = a.text
                db = a.get("key")
                isni = a.get("isni")
                orcid = a.get("orcid")
                viaf = a.get("viaf")
                orkg = a.get("orkg")
                gnd = a.get("gnd")
                wikidata = a.get("wikidata")

                if name:
                    authors.append(name)
                    dblp.append(db or None)
                    isnis.append(isni or None)
                    orcids.append(orcid or None)
                    viafs.append(viaf or None)
                    orkgs.append(orkg or None)
                    gnds.append(gnd or None)
                    wikis.append(wikidata or None)

            articles.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": "; ".join(authors),
                "dblp": "; ".join([p for p in dblp if p]),
                "isnis": "; ".join([i for i in isnis if i]),
                "orcids": "; ".join([o for o in orcids if o]),
                "viafs": "; ".join([v for v in viafs if v]),
                "orkgs": "; ".join([o for o in orkgs if o]),
                "gnds": "; ".join([g for g in gnds if g]),
                "wikidata": "; ".join([w for w in wikis if w]),
                "doi": doi
            })

            elem.clear()

df = pd.DataFrame(articles)

In [4]:
id_cols = ["pids"]

mask = df[id_cols].notna().any(axis=1) & (df[id_cols].astype(str).apply(lambda x: x.str.strip() != "").any(axis=1))
df[mask]

,title,journal,year,authors,pids,isnis,orcids,viafs,orkgs,gnds,wikidata,doi


In [5]:
df["dblp"]

0           
1           
2           
3           
4           
          ..
3992218     
3992219     
3992220     
3992221     
3992222     
Name: dblp, Length: 3992223, dtype: object

In [ ]:
records = []

# Tags we want to process
tags = ["inproceedings", "article"]

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        # iterparse supports only one tag at a time, so we'll parse for both sequentially
        for tag in tags:
            context = etree.iterparse(gz, events=("end",), tag=tag, recover=True)
            for event, elem in context:
                title = elem.findtext("title")

                # Extract DOIs from <ee> elements
                dois = [ee.text for ee in elem.findall(".//ee") if ee.text and "doi.org" in ee.text]
                doi_str = "; ".join(dois) if dois else None

                if title and doi_str:  # keep only entries that have both
                    records.append({
                        "title": title,
                        "dois": doi_str
                    })

                elem.clear()
            # Reset the file pointer for the next tag
            gz.seek(0)

ERROR! Session/line number was not unique in database. History logging moved to new session 9


In [2]:
articles = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="article", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("year")
            doi = elem.findtext("ee")
            

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            articles.append({
                "title": title,
                "journal": journal,
                "year": pub_date,
                "authors": authors_str,
                "doi":doi
            })

            elem.clear()

df = pd.DataFrame(articles)

In [2]:
inpro = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            conf = elem.findtext("booktitle")
            pub_date = elem.findtext("year")
            doi = elem.findtext("ee")

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            inpro.append({
                "title": title,
                "conf": conf,
                "year": pub_date,
                "authors": authors_str,
                "doi":doi
            })

            elem.clear()

df = pd.DataFrame(inpro)

In [9]:

import re
df[df["authors"].apply(lambda x: bool(re.compile(r"^[A-Za-z\s-]+$").search(str(x))))]


,title,conf,year,authors
0,The Future of Classic Data Administration: Obj...,SWEE,1998,Arnon Rosenthal
1,Some Patterns of Convincing Software Engineeri...,Denert Award,2020,Lutz Prechelt
2,Static Worst-Case Analyses and Their Validatio...,Denert Award,2020,Peter Wgemann
3,Crossing Disciplinary Borders to Improve Requi...,Denert Award,2020,Anne Hess
4,What You See Is What You Get: Practical Effect...,Denert Award,2020,Jonathan Immanuel Brachthuser
...,...,...,...,...
3749504,"Machine Intentionality, the Moral Status of Ma...",PT-AI,2011,David Leech Anderson
3749509,Practical Introspection as Inspiration for AI.,PT-AI,2011,Sam Freed
3749510,"""Computational Ontology and Deontology"".",PT-AI,2011,Raffaela Giovagnoli
3749512,Emotional Control-Conditio Sine Qua Non for Ad...,PT-AI,2011,Claudius Gros


In [2]:
from lxml import etree
import gzip

direct_set = set()

with fs.open(path, "rb") as f:
    with gzip.GzipFile(fileobj=f) as gz:
        context = etree.iterparse(gz, events=("end",), tag="inproceedings", recover=True)
        for event, elem in context:
            # collect child tags
            for child in elem:
                if child.tag is not None:
                    direct_set.add(child.tag)

            # clear memory: remove element and its previous siblings
            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]

print(sorted(direct_set))

['author', 'booktitle', 'cdrom', 'cite', 'crossref', 'editor', 'ee', 'month', 'note', 'number', 'pages', 'publnr', 'stream', 'title', 'url', 'volume', 'year']


In [30]:
chunks = []
n_chunks = 50
chunk_size = len(df) // n_chunks

for i in range(n_chunks):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else len(df)  # last chunk includes remainder
    
    chunk = df.iloc[start:end].copy()
    
    # Split authors column into multiple columns
    #authors_df = chunk['authors'].str.split(';', expand=True)
    #authors_df.columns = [f'author{i+1}' for i in range(authors_df.shape[1])]
    
    # Combine with original chunk
    #chunk = pd.concat([chunk.drop(columns=['authors']), authors_df], axis=1)
    
    chunk.to_parquet(f"inpro/inproceedings_chunk_{i}.parquet", index=False)

NameError: name 'df' is not defined

In [35]:

import pandas as pd
import re
from collections import defaultdict
import glob

ego_data = defaultdict(lambda: {"publications": []}) #set permet de créer des clefs si absentes et de les actualiser si présentes (en définissant la classe des clefs)

chunk_files = glob.glob("inpro/inproceedings_chunk_*.parquet")

for file in chunk_files:
    df_chunk = pd.read_parquet(file)
    for _, row in df_chunk.iterrows():
        authors = row['authors'].split(';')
        #authors = [a for a in authors if not re.compile(r"\d").search(a)]
        for ego in authors:
            coauthors = set(authors) - {ego}
            
            ego_data[ego]["publications"].append({
                "coauthors": list(coauthors),
                "source": row.get("conf"),
                "year": row.get("year"),
                "title": row.get("title"),
                "doi": row.get("doi")
            })

In [36]:
rows = []
for ego, data in ego_data.items():
    for pub in data["publications"]:
        rows.append({
            "ego": ego,
            "coauthors": ",".join(sorted(pub["coauthors"])),
            "source": pub.get("source"),
            "year": pub.get("year"),
            "title": pub.get("title"),
            "doi": pub.get("doi")
        })

inp_df = pd.DataFrame(rows)


In [37]:
inp_df

,ego,coauthors,source,year,title,doi
0,Sanish Rai,Xiaolin Hu 0002,SpringSim (TMS),2018,Hybrid agent-based and graph-based modeling fo...,http://dl.acm.org/citation.cfm?id=3213189
1,Sanish Rai,Xiaolin Hu 0002,IAT,2013,Behavior Pattern Detection for Data Assimilati...,https://doi.org/10.1109/WI-IAT.2013.106
2,Sanish Rai,"Minghao Wang, Xiaolin Hu 0002",SpringSim (ADS),2015,A graph-based agent-oriented model for buildin...,http://dl.acm.org/citation.cfm?id=2872548
3,Sanish Rai,Xiaolin Hu 0002,WSC,2017,Data assimilation with sensor-informed resampl...,https://doi.org/10.1109/WSC.2017.8247862
4,Sanish Rai,Xiaolin Hu 0002,COMPSAC,2016,Graph Based Agent Oriented Model for Tunnel Si...,https://doi.org/10.1109/COMPSAC.2016.223
...,...,...,...,...,...,...
12855429,Juan Gutierrez,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444
12855430,Ashley Houston-King,"Cliff Freeman, Eli Tucker-Raymond, Katherine ...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444
12855431,Cliff Freeman,"Ashley Houston-King, Eli Tucker-Raymond, Kath...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444
12855432,Katherine K. Frankel,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444


In [38]:
art_df["Type"]="art"
inp_df["Type"]="inp"
ego_df=pd.concat([art_df,inp_df], axis=0, ignore_index=True)
ego_df

,ego,coauthors,source,year,title,doi,Type
0,Kodai Shimosato,Norimichi Ukita,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
1,Kodai Shimosato,Norimichi Ukita,IEEE Access,2021,Multi-Modal Data Fusion for Land-Subsidence Im...,https://doi.org/10.1109/ACCESS.2021.3120133,art
2,Norimichi Ukita,Kodai Shimosato,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
3,Norimichi Ukita,"Arnau Raventos, Aryan Esfandiari, C. Victor J...",CoRR,2020,AIM 2020 Challenge on Video Extreme Super-Reso...,https://arxiv.org/abs/2009.06290,art
4,Norimichi Ukita,"Kazutoshi Akita,Kyotaro Tokoro",CoRR,2024,Burst Super-Resolution with Diffusion Models f...,https://doi.org/10.48550/arXiv.2403.19428,art
...,...,...,...,...,...,...,...
26701427,Juan Gutierrez,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp
26701428,Ashley Houston-King,"Cliff Freeman, Eli Tucker-Raymond, Katherine ...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp
26701429,Cliff Freeman,"Ashley Houston-King, Eli Tucker-Raymond, Kath...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp
26701430,Katherine K. Frankel,"Ashley Houston-King, Cliff Freeman, Eli Tucke...",FabLearn/Constructionism,2023,Youth Pedagogical Development in Youth Teachin...,https://doi.org/10.1145/3615430.3615444,inp


In [39]:
#seek coauthors that do not appear as egos

ego_set = set(ego_df['ego'])

coauthor_set = set()
for coauthor_str in ego_df['coauthors']:
    if pd.notna(coauthor_str) and coauthor_str != '':
        coauthor_set.update([c.strip() for c in coauthor_str.split(',')])

missing_egos = coauthor_set - ego_set

if missing_egos:
    new_rows = pd.DataFrame({
        'ego': list(missing_egos),
        'coauthors': [''] * len(missing_egos)
    })
    ego_df = pd.concat([ego_df, new_rows], ignore_index=True)

ego_df

,ego,coauthors,source,year,title,doi,Type
0,Kodai Shimosato,Norimichi Ukita,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
1,Kodai Shimosato,Norimichi Ukita,IEEE Access,2021,Multi-Modal Data Fusion for Land-Subsidence Im...,https://doi.org/10.1109/ACCESS.2021.3120133,art
2,Norimichi Ukita,Kodai Shimosato,CoRR,2024,Inpainting-Driven Mask Optimization for Object...,https://doi.org/10.48550/arXiv.2403.15849,art
3,Norimichi Ukita,"Arnau Raventos, Aryan Esfandiari, C. Victor J...",CoRR,2020,AIM 2020 Challenge on Video Extreme Super-Reso...,https://arxiv.org/abs/2009.06290,art
4,Norimichi Ukita,"Kazutoshi Akita,Kyotaro Tokoro",CoRR,2024,Burst Super-Resolution with Diffusion Models f...,https://doi.org/10.48550/arXiv.2403.19428,art
...,...,...,...,...,...,...,...
28630100,Geetanjali R. Kamath,,NaN,NaN,NaN,NaN,NaN
28630101,Shigeaki Matsuoka,,NaN,NaN,NaN,NaN,NaN
28630102,Martin Koppers,,NaN,NaN,NaN,NaN,NaN
28630103,Ehren Dohler,,NaN,NaN,NaN,NaN,NaN


In [ ]:
import re

ego_df["ego_number"]=ego_df["ego"].apply(lambda x: re.findall(r'\d+',x))
ego_df["ego_number"]=ego_df["ego_number"].str[0]
ego_df["ego"]=ego_df["ego"].apply(lambda x: re.sub(r"\d","",x))
ego_df["name"]=ego_df["ego"].str.lstrip().str.split(" ").str[0].str.lower()
ego_df[~ego_df["ego_number"].isna()]

,ego,coauthors,source,year,title,doi,Type,ego_number
65,Zheyuan Hu,"Ameya D. Jagtap, George Em Karniadakis, Kenji...",CoRR,2021,When Do Extended Physics-Informed Neural Netwo...,https://arxiv.org/abs/2109.09444,art,0002
66,Zheyuan Hu,"George Em Karniadakis, Kenji Kawaguchi, Khemr...",CoRR,2023,Tackling the Curse of Dimensionality with Phys...,https://doi.org/10.48550/arXiv.2307.12306,art,0002
67,Zheyuan Hu,"Ameya D. Jagtap, George Em Karniadakis, Kenji...",CoRR,2022,Augmented Physics-Informed Neural Networks (AP...,https://doi.org/10.48550/arXiv.2211.08939,art,0002
68,Zheyuan Hu,"Ameya D. Jagtap, George Em Karniadakis, Kenji...",SIAM J. Sci. Comput.,2022,When Do Extended Physics-Informed Neural Netwo...,https://doi.org/10.1137/21m1447039,art,0002
69,Zheyuan Hu,"George E. Karniadakis, Kenji Kawaguchi, Zhong...",SIAM J. Sci. Comput.,2025,Score-Based Physics-Informed Neural Networks f...,https://doi.org/10.1137/24m1638768,art,0002
...,...,...,...,...,...,...,...,...
28629841,Jun Cui,,NaN,NaN,NaN,NaN,NaN,0002
28629859,Jianfeng Liu,,NaN,NaN,NaN,NaN,NaN,0003
28629942,Kun Zhao,,NaN,NaN,NaN,NaN,NaN,0011
28630016,James Chapman,,NaN,NaN,NaN,NaN,NaN,0003


In [47]:

with fs.open("s3://lab/main_df.parquet", 'wb') as f:
    ego_df.to_parquet(f, index=False)

In [27]:
import numpy as np
ego_df["name"]=np.where(len(ego_df["ego"].str.split())>1, ego_df["ego"].str.split(n=1).str[0], None)
def is_valid_firstname(word):
    if not isinstance(word, str):  # skip NaN/None
        return False
    if len(word) == 1:  # single letter
        return False
    if word.isupper():  # all caps (acronyms)
        return False
    if not re.match(r'^[A-Za-z-]+$', word):  # contains only letters or hyphens
        return False
    return True

ego_df["name"] = ego_df["name"].apply(lambda x: x if is_valid_firstname(x) else None)


In [1]:
ego_df["name"]=ego_df["ego"].str.lstrip().str.split(" ").str[0].str.lower()


NameError: name 'ego_df' is not defined

In [32]:
names=names.drop_duplicates()

In [8]:
ego_df.to_parquet("ego_df.parquet",index=False)


In [19]:
with fs.open("s3://lab/name_gender_dataset.csv") as f:
    g1 = pd.read_csv(f)
with fs.open("s3://lab/wgnd_2_0_name-gender-langcode.csv") as f:
    g2 = pd.read_csv(f)

g1=g1[["Name","Gender"]]
g2[["Name","Gender"]]=g2[["name","gender"]]
g2=g2[["Name","Gender"]]

In [20]:
g2[100:150]

,Name,Gender
100,a laura,F
101,a lelia,F
102,a lelia,F
103,a lerah,F
104,a lerah,F
105,a lex ander,M
106,a lex ander,M
107,a lie,M
108,a lisa,F
109,a lisa,F


In [22]:
pd.concat([g1,g2],axis=0,ignore_index=True)

,Name,Gender
0,James,M
1,John,M
2,Robert,M
3,Michael,M
4,William,M
...,...,...
21978307,히카리,F
21978308,히토미,F
21978309,힘찬,M
21978310,凉峰,M
